<a href="https://colab.research.google.com/github/phonemyat-4116/SentimentAnalysisPrj/blob/roberta-sentiment/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [2]:
!ngrok authtoken 2uGIP7ca9LpQAHmxHGxAn68wjby_6dsq5AnRK8MyVkNkfo65L

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
%%writefile app.py
# import streamlit as st
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from transformers import pipeline, RobertaTokenizer
# import torch


# # Check if CUDA is available and set device accordingly
# device = 0 if torch.cuda.is_available() else -1

# # Initialize sentiment analysis pipeline with RoBERTa
# @st.cache_resource
# def load_sentiment_model():
#     return pipeline(
#         "sentiment-analysis",
#         model="cardiffnlp/twitter-roberta-base-sentiment",
#         return_all_scores=True,  # Return scores for all sentiment classes
#         device=device  # Use GPU if available
#     )

# sentiment_pipeline = load_sentiment_model()

# # Initialize the tokenizer
# tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

# def analyze_sentiment(text):
#     """Analyze sentiment using RoBERTa."""
#     max_length = 512  # RoBERTa's max token length
#     # Tokenize the text and check its length
#     tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

#     # If the token length exceeds the limit, chunk the text
#     if len(tokens['input_ids'][0]) > max_length:
#         # Split the text into chunks based on token length
#         chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
#         results = []
#         for chunk in chunks:
#             results.append(sentiment_pipeline(chunk)[0])

#         # Average the scores across chunks
#         neg = sum(r[0]['score'] for r in results) / len(results)
#         neu = sum(r[1]['score'] for r in results) / len(results)
#         pos = sum(r[2]['score'] for r in results) / len(results)
#     else:
#         # Process the text if it fits within the token limit
#         result = sentiment_pipeline(text)[0]
#         neg = result[0]['score']
#         neu = result[1]['score']
#         pos = result[2]['score']

#     compound = pos - neg

#     return {
#         'pos': pos,
#         'neg': neg,
#         'neu': neu,
#         'compound': compound
#     }

# def process_uploaded_file(uploaded_file):
#     """Process uploaded CSV file and analyze sentiment."""
#     df = pd.read_csv(uploaded_file)

#     # Determine which column to use for sentiment analysis
#     valid_columns = ['Feedback', 'Review', 'Text']
#     selected_column = next((col for col in valid_columns if col in df.columns), None)

#     if not selected_column:
#         st.error("CSV must contain a 'Feedback', 'Review', or 'Text' column.")
#         return None

#     # Create a progress bar
#     progress_bar = st.progress(0)
#     total_rows = len(df)

#     # Apply sentiment analysis with progress updates
#     results = []
#     for i, text in enumerate(df[selected_column]):
#         scores = analyze_sentiment(str(text))
#         results.append(scores)
#         # Update progress bar
#         progress_bar.progress((i + 1) / total_rows)

#     # Convert results to DataFrame and join with original df
#     scores_df = pd.DataFrame(results)
#     df[['positive', 'negative', 'neutral', 'compound']] = scores_df[['pos', 'neg', 'neu', 'compound']]

#     # Categorize sentiment
#     df['Sentiment'] = df.apply(lambda row: "Positive" if row['compound'] > 0.05
#                               else "Negative" if row['compound'] < -0.05 else "Neutral", axis=1)

#     # Remove progress bar when done
#     progress_bar.empty()

#     return df

# def plot_sentiment_charts(df):
#     """Plot sentiment distribution as both a pie chart and a bar chart."""
#     sentiment_counts = df['Sentiment'].value_counts()

#     labels = sentiment_counts.index.tolist()
#     sizes = sentiment_counts.values.tolist()
#     colors = {'positive': '#3bed68', 'negative': '#ed184d', 'neutral': '#1776eb'}

#     # Convert labels to lowercase to match the keys in the colors dictionary
#     color_list = [colors[label.lower()] for label in labels]

#     # Create layout for two charts
#     col1, col2 = st.columns(2)

#     # Pie Chart
#     with col1:
#         plt.figure(figsize=(5, 5))
#         plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=color_list, startangle=140)
#         plt.title('Sentiment Distribution')
#         st.pyplot(plt)

#     # Bar Chart
#     with col2:
#         plt.figure(figsize=(4.5, 5))
#         sns.barplot(x=labels, y=sizes, palette=color_list)
#         plt.ylabel("Count")
#         plt.title("Sentiment Distribution")
#         st.pyplot(plt)

#     # Summary Text
#     dominant_sentiment = sentiment_counts.idxmax()
#     st.write(f"### Summary: The overall sentiment of the data is **{dominant_sentiment}**.")

# # Streamlit UI
# st.title("Advanced Sentiment Analysis with RoBERTa")
# st.markdown("""
# This app uses the **cardiffnlp/twitter-roberta-base-sentiment** model,
# which performs better on longer texts than VADER.
# """)

# # Tabs for different functionalities
# tabs = st.tabs(["Upload CSV", "Text Input"])

# # Part 1: File Upload Sentiment Analysis
# with tabs[0]:
#     st.header("Upload CSV for Sentiment Analysis")
#     uploaded_file = st.file_uploader("Upload a CSV file with a 'Feedback', 'Review', or 'Text' column", type=["csv"])

#     if uploaded_file:
#         with st.spinner("Analyzing sentiment... This may take a moment for large files."):
#             df_result = process_uploaded_file(uploaded_file)

#         if df_result is not None:
#             st.write("### Sentiment Analysis Results")

#             # Display tables for each sentiment category
#             for sentiment in ['Positive', 'Negative', 'Neutral']:
#                 # Determine which column to display
#                 display_column = next((col for col in ['Review', 'Feedback', 'Text'] if col in df_result.columns), None)

#                 if display_column:
#                     filtered_df = df_result[df_result['Sentiment'] == sentiment][[display_column, 'Sentiment', 'positive', 'negative', 'neutral']]
#                     if not filtered_df.empty:
#                         st.write(f"### {sentiment} Reviews ({len(filtered_df)} total)")
#                         st.dataframe(filtered_df.reset_index(drop=True))

#             plot_sentiment_charts(df_result)

#             # Option to download the analyzed data
#             csv = df_result.to_csv(index=False)
#             st.download_button(
#                 label="Download analyzed data as CSV",
#                 data=csv,
#                 file_name="sentiment_analysis_results.csv",
#                 mime="text/csv",
#             )

# # Part 2: Text Input Sentiment Analysis
# with tabs[1]:
#     st.header("Type Your Text for Sentiment Analysis")
#     user_text = st.text_area("Enter text here (works well with longer reviews):", height=200)

#     if st.button("Calculate", help="Click to analyze sentiment"):
#         if user_text.strip():  # Ensure input is not empty
#             with st.spinner("Analyzing sentiment..."):
#                 result = analyze_sentiment(user_text)

#             st.write("### Sentiment Scores")

#             def sentiment_bar(label, value, color):
#                 """Function to create a sentiment bar visualization"""
#                 bar_html = f"""
#                 <div style="margin-bottom: 1rem">
#                     <div style="width: {value*100}%; height: 10px; background-color: {color}; padding: 2px; border-radius: 5px; text-align: center; color: white;">
#                     </div>
#                     {label}: {value:.3f}
#                 </div>
#                 """
#                 st.markdown(bar_html, unsafe_allow_html=True)

#             # Displaying sentiment bars
#             sentiment_bar("Negative", result['neg'], "#ed187b")
#             sentiment_bar("Neutral", result['neu'], "#1776eb")
#             sentiment_bar("Positive", result['pos'], "#3bed68")

#             # Displaying compound score
#             compound_score = result['compound']
#             sentiment_label = "Positive" if compound_score > 0.05 else "Negative" if compound_score < -0.05 else "Neutral"
#             st.write(f"Compound Score: {compound_score:.4f}")
#             st.write(f"### Overall Sentiment: **{sentiment_label}**")
#         else:
#             st.warning("Please enter some text before calculating.")

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline, RobertaTokenizer
import torch
from concurrent.futures import ThreadPoolExecutor

# Check if CUDA is available and set device accordingly
device = 0 if torch.cuda.is_available() else -1

# 1. Optimized Model Initialization with proper caching
@st.cache_resource
def load_sentiment_model():
    tokenizer = RobertaTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
    model = pipeline(
        "sentiment-analysis",
        model="cardiffnlp/twitter-roberta-base-sentiment",
        return_all_scores=True,
        device=device
    )
    return model, tokenizer

# Load model and tokenizer once
sentiment_pipeline, tokenizer = load_sentiment_model()

def analyze_sentiment(text):
    """Analyze sentiment using RoBERTa."""
    max_length = 512  # RoBERTa's max token length
    # Tokenize the text and check its length
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    # If the token length exceeds the limit, chunk the text
    if len(tokens['input_ids'][0]) > max_length:
        # Split the text into chunks based on token length
        chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
        results = []
        for chunk in chunks:
            results.append(sentiment_pipeline(chunk)[0])

        # Average the scores across chunks
        neg = sum(r[0]['score'] for r in results) / len(results)
        neu = sum(r[1]['score'] for r in results) / len(results)
        pos = sum(r[2]['score'] for r in results) / len(results)
    else:
        # Process the text if it fits within the token limit
        result = sentiment_pipeline(text)[0]
        neg = result[0]['score']
        neu = result[1]['score']
        pos = result[2]['score']

    compound = pos - neg

    return {
        'pos': pos,
        'neg': neg,
        'neu': neu,
        'compound': compound
    }

# 2. Batch processing implementation
def process_uploaded_file(uploaded_file):
    """Process uploaded CSV file and analyze sentiment using parallel processing."""
    with st.spinner("Loading file..."):
        df = pd.read_csv(uploaded_file)

    # Determine which column to use for sentiment analysis
    valid_columns = ['Feedback', 'Review', 'Text']
    selected_column = next((col for col in valid_columns if col in df.columns), None)

    if not selected_column:
        st.error("CSV must contain a 'Feedback', 'Review', or 'Text' column.")
        return None

    total_rows = len(df)
    st.info(f"File loaded: {total_rows} rows")

    # Create a progress bar
    progress_bar = st.progress(0)
    status_text = st.empty()

    # Prepare texts as strings
    texts = [str(text) for text in df[selected_column]]

    # Determine optimal approach based on size
    if total_rows > 500:  # For larger datasets
        # Use parallel processing with ThreadPoolExecutor
        results = []
        batch_size = 20  # Process in batches for better progress tracking

        with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust max_workers based on your system
            for i in range(0, total_rows, batch_size):
                end_idx = min(i + batch_size, total_rows)
                batch_texts = texts[i:end_idx]

                # Process batch in parallel
                batch_results = list(executor.map(analyze_sentiment, batch_texts))
                results.extend(batch_results)

                # Update progress
                progress = min(1.0, end_idx / total_rows)
                progress_bar.progress(progress)
                status_text.text(f"Processing rows {i+1}-{end_idx} of {total_rows}")
    else:
        # For smaller datasets, simpler processing is fine
        results = []
        for i, text in enumerate(texts):
            results.append(analyze_sentiment(text))
            # Update progress
            progress_bar.progress((i + 1) / total_rows)
            status_text.text(f"Processing row {i+1}/{total_rows}")

    # Convert results to DataFrame and join with original df
    scores_df = pd.DataFrame(results)
    df[['positive', 'negative', 'neutral', 'compound']] = scores_df[['pos', 'neg', 'neu', 'compound']]

    # Categorize sentiment
    df['Sentiment'] = df.apply(lambda row: "Positive" if row['compound'] > 0.05
                              else "Negative" if row['compound'] < -0.05 else "Neutral", axis=1)

    # Remove progress elements when done
    progress_bar.empty()
    status_text.empty()

    return df

def plot_sentiment_charts(df):
    """Plot sentiment distribution as both a pie chart and a bar chart."""
    sentiment_counts = df['Sentiment'].value_counts()

    labels = sentiment_counts.index.tolist()
    sizes = sentiment_counts.values.tolist()
    colors = {'positive': '#3bed68', 'negative': '#ed184d', 'neutral': '#1776eb'}

    # Convert labels to lowercase to match the keys in the colors dictionary
    color_list = [colors[label.lower()] for label in labels]

    # Create layout for two charts
    col1, col2 = st.columns(2)

    # Pie Chart
    with col1:
        plt.figure(figsize=(5, 5))
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=color_list, startangle=140)
        plt.title('Sentiment Distribution')
        st.pyplot(plt)

    # Bar Chart
    with col2:
        plt.figure(figsize=(4.5, 5))
        sns.barplot(x=labels, y=sizes, palette=color_list)
        plt.ylabel("Count")
        plt.title("Sentiment Distribution")
        st.pyplot(plt)

    # Summary Text
    dominant_sentiment = sentiment_counts.idxmax()
    st.write(f"### Summary: The overall sentiment of the data is **{dominant_sentiment}**.")

# Streamlit UI
st.title("Advanced Sentiment Analysis with RoBERTa")
st.markdown("""
This app uses the **cardiffnlp/twitter-roberta-base-sentiment** model,
which performs better on longer texts than VADER.
""")

# Tabs for different functionalities
tabs = st.tabs(["Upload CSV", "Text Input"])

# Part 1: File Upload Sentiment Analysis
with tabs[0]:
    st.header("Upload CSV for Sentiment Analysis")
    uploaded_file = st.file_uploader("Upload a CSV file with a 'Feedback', 'Review', or 'Text' column", type=["csv"])

    if uploaded_file:
        with st.spinner("Analyzing sentiment... This may take a moment for large files."):
            df_result = process_uploaded_file(uploaded_file)

        if df_result is not None:
            st.write("### Sentiment Analysis Results")

            # Display tables for each sentiment category
            for sentiment in ['Positive', 'Negative', 'Neutral']:
                # Determine which column to display
                display_column = next((col for col in ['Review', 'Feedback', 'Text'] if col in df_result.columns), None)

                if display_column:
                    filtered_df = df_result[df_result['Sentiment'] == sentiment][[display_column, 'Sentiment', 'positive', 'negative', 'neutral']]
                    if not filtered_df.empty:
                        st.write(f"### {sentiment} Reviews ({len(filtered_df)} total)")
                        st.dataframe(filtered_df.reset_index(drop=True))

            plot_sentiment_charts(df_result)

            # Option to download the analyzed data
            csv = df_result.to_csv(index=False)
            st.download_button(
                label="Download analyzed data as CSV",
                data=csv,
                file_name="sentiment_analysis_results.csv",
                mime="text/csv",
            )

# Part 2: Text Input Sentiment Analysis
with tabs[1]:
    st.header("Type Your Text for Sentiment Analysis")
    user_text = st.text_area("Enter text here (works well with longer reviews):", height=200)

    if st.button("Calculate", help="Click to analyze sentiment"):
        if user_text.strip():  # Ensure input is not empty
            with st.spinner("Analyzing sentiment..."):
                result = analyze_sentiment(user_text)

            st.write("### Sentiment Scores")

            def sentiment_bar(label, value, color):
                """Function to create a sentiment bar visualization"""
                bar_html = f"""
                <div style="margin-bottom: 1rem">
                    <div style="width: {value*100}%; height: 10px; background-color: {color}; padding: 2px; border-radius: 5px; text-align: center; color: white;">
                    </div>
                    {label}: {value:.3f}
                </div>
                """
                st.markdown(bar_html, unsafe_allow_html=True)

            # Displaying sentiment bars
            sentiment_bar("Negative", result['neg'], "#ed187b")
            sentiment_bar("Neutral", result['neu'], "#1776eb")
            sentiment_bar("Positive", result['pos'], "#3bed68")

            # Displaying compound score
            compound_score = result['compound']
            sentiment_label = "Positive" if compound_score > 0.05 else "Negative" if compound_score < -0.05 else "Neutral"
            st.write(f"Compound Score: {compound_score:.4f}")
            st.write(f"### Overall Sentiment: **{sentiment_label}**")
        else:
            st.warning("Please enter some text before calculating.")



Overwriting app.py


In [4]:
!ngrok tunnels

ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions
  Upgrade your account at https://dash

In [9]:
!pkill -f ngrok


In [10]:
import os
import time
from pyngrok import ngrok

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Wait for the server to start
time.sleep(5)

# Create Ngrok tunnel
public_url = ngrok.connect(8501)
print("Streamlit app is live at:", public_url)


Streamlit app is live at: NgrokTunnel: "https://f8d9-34-16-205-121.ngrok-free.app" -> "http://localhost:8501"
